In [1]:
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
from subprocess import check_output
from seedlings_resnet_50 import resnet50_model

from PIL import Image
%matplotlib inline


Using TensorFlow backend.


In [2]:
fpa = []
fpa.append(check_output(["ls", "seedlings_data/test/"]).decode("utf8").strip().split("\n"))

filez = np.array(fpa)

fpa = np.array(fpa)
fpa = np.core.defchararray.add("seedlings_data/test/", fpa)
fpa = fpa.squeeze()
fpa.shape

(794,)

In [3]:
imgs = []

for im_path in fpa:
    if 'Thumbs.db' not in im_path:
        #print(im_path)
        imageCV = cv2.imread(im_path)
        imgs.append(cv2.resize(imageCV, (300, 300), interpolation=cv2.INTER_AREA))

imgs = np.array(imgs)
imgs.shape

(794, 300, 300, 3)

In [4]:
classes = check_output(["ls", "seedlings_data/train"]).decode("utf8").strip().split("\n")

In [5]:
img_rows, img_cols = 300, 300  # Resolution of inputs
channel = 3
num_classes = 12
batch_size = 32
epochs = 2

In [6]:
submModel = resnet50_model(img_rows, img_cols, channel, num_classes)
preds = submModel.predict(imgs, batch_size=32, verbose=1)
print(preds.shape)
preds

Q:\00_DEV\projects\cnn_finetune\seedlings_resnet_50.py:82: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, shortcut], mode='sum')
C:\Users\michal\Anaconda3\envs\tensorf\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Q:\00_DEV\projects\cnn_finetune\seedlings_resnet_50.py:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, input_tensor], mode='sum')


Dimeansion ordering (tf for Tensorflow): tf
Loading weights from file: seedl_chkp/seedlings_weigts_imp-01-0.98.hdf5


Loading weights finished...


 32/794 [>.............................] - ETA: 5:05

...

736/794 [==========================>...] - ETA: 23s

768/794 [============================>.] - ETA: 10s

794/794 [==============================] - 316s 398ms/step


(794, 12)


array([[  4.42441582e-04,   2.68338161e-04,   7.21549382e-04, ...,
          3.13883205e-03,   9.91815031e-01,   1.12224981e-04],
       [  4.35414950e-05,   6.34877688e-06,   7.46459264e-05, ...,
          7.24479105e-05,   2.37629447e-05,   1.65545440e-04],
       [  1.84478449e-05,   3.98467710e-05,   8.67307608e-05, ...,
          8.61632259e-07,   9.88240231e-07,   9.99180377e-01],
       ..., 
       [  6.12012518e-05,   2.68013740e-04,   3.86299158e-04, ...,
          2.85143942e-05,   1.05075715e-05,   9.98283625e-01],
       [  1.37131638e-05,   9.98666406e-01,   1.35354014e-04, ...,
          3.11321310e-05,   6.06936592e-05,   1.00642217e-04],
       [  1.00143723e-01,   1.18362797e-04,   6.68600318e-04, ...,
          1.07497144e-05,   2.82710069e-04,   7.36038128e-05]], dtype=float32)

In [7]:
classes = np.array(classes)
classes.shape
max_idxes = np.argmax(preds, axis=1)
max_idxes.shape

pred_class = classes[max_idxes]
pred_class = np.expand_dims(pred_class, axis=0)
print(pred_class.shape)
print(filez.shape)

conc = np.concatenate((filez, pred_class), axis=0)
conc = conc.astype('str')
print(conc.shape)
conc


(1, 794)
(1, 794)
(2, 794)


array([['0021e90e4.png', '003d61042.png', '007b3da8b.png', ...,
        'fef2ade8c.png', 'ff65bc002.png', 'ffc6f8527.png'],
       ['Small-flowered Cranesbill', 'Fat Hen', 'Sugar beet', ...,
        'Sugar beet', 'Charlock', 'Common wheat']],
      dtype='<U25')

In [8]:
res = np.apply_along_axis(lambda d: d[0] + ',' + d[1], 0, conc)

res.tofile("seedl_subms/subm94.csv", sep='\n')
#np.savetxt("seedl_subms/subm94.csv", conc, delimiter=",")